In [5]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import Matrix, symbols
from scipy.integrate import nquad
from concurrent.futures import ProcessPoolExecutor
!pip install vegas
import vegas

!pip install pyscf
from pyscf import gto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 11.4 MB/s eta 0:00:00


#1. Define atomic coordinates, nuclear charges and orbitals

In [6]:
## Molecule (e.g. water)

# N.B. distances are in Bohr radii (this will produce a final ground state energy in Hartrees)

Molecule = {
    "Atom_label": ["O", "H", "H"],
    "Z": [8, 1, 1],
    "x": [0.00000, 0.00000, 0.00000],
    "y": [0.00000, 1.42757, -1.42757],
    "z": [0.00000, -1.11294, -1.11294]
}

df = pd.DataFrame(Molecule)
print(df)


##Orbitals

#Define symbolic coordinates for integration via Sympy

x, y, z = sp.symbols('x y z')
r = (x, y, z)

def sto_3G_1s(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2*alphas[i])/sp.pi)**0.75 * sp.exp(-alphas[i] *((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2s(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2*alphas[i])/sp.pi)**0.75 * sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2px(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*(x - Rx)*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2py(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*(y - Ry)*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2pz(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*((z - Rz))*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

n_electrons = sum(Molecule["Z"])
print("number of electrons =", n_electrons)

basis_set = [sto_3G_1s(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [130.709321, 23.80886605, 6.44360831]),
             sto_3G_2s(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [-0.0999672, 0.39951282, 0.70011546], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2px(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2py(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2pz(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_1s(r, center=(Molecule['x'][1], Molecule['y'][1], Molecule['z'][1]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [3.4252509, 0.62391372, 0.16885540]),
             sto_3G_1s(r, center=(Molecule['x'][2], Molecule['y'][2], Molecule['z'][2]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [3.4252509, 0.62391372, 0.16885540])
]

# N.B. Approximate alpha coefficients taken from https://www.basissetexchange.org/

  Atom_label  Z    x        y        z
0          O  8  0.0  0.00000  0.00000
1          H  1  0.0  1.42757 -1.11294
2          H  1  0.0 -1.42757 -1.11294
number of electrons = 10


#2. Calculating one-electron integrals of the Fock matrix [H_core = T + V_{e-nuc}]

In [ ]:
#Constructing T matrix
#precompute laplacians for efficiency

x, y, z, alpha, Rx, Ry, Rz = sp.symbols('x y z alpha Rx Ry Rz')
laplacians = [sp.diff(phi, x, 2) + sp.diff(phi, y, 2) + sp.diff(phi, z, 2) for phi in basis_set]



def t_ij(i, j, basis_set, laplacians):
  integrand = sp.lambdify([[x, y, z]], -0.5 * basis_set[i] * laplacians[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  t = integrator(integrand, nitn=50, neval=10000).mean
  return t


#Parallelisation needed to compute 7x7 matrix within a few mins

def parallel_T_matrix(basis_set, laplacians):
    n = len(basis_set)
    T = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(t_ij, i, j, basis_set, laplacians) for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                T[i, j] = futures[i][j].result()
        # Calculate Oxygen 1s KE more accurately since numeric integration struggles to capture it
        integrand = sp.lambdify((x, y, z), - 0.5 * basis_set[0] * laplacians[0], modules='numpy')
        T[1, 1], integration_error = nquad(integrand, [[-2, 2], [-2, 2], [-2, 2]])
    return T

T = parallel_T_matrix(basis_set, laplacians)

print(T)

In [8]:
#Constructing V matrix
x, y, z, alpha, Rx, Ry, Rz = sp.symbols('x y z alpha Rx Ry Rz')


#constructing V_func = -Sigma(Z/abs(r - R)) term:
V_funcs = 0
for i in range(len(Molecule["Z"])):
  V_func = - Molecule["Z"][i] / (sp.sqrt((x - Molecule["x"][i])**2 + (y - Molecule["y"][i])**2 + (z - Molecule["z"][i])**2))
  V_funcs += V_func


#precomuting integrands
integrands = [[sp.lambdify((x, y, z), V_funcs * basis_set[i] * basis_set[j], modules='numpy')
              for j in range(len(basis_set))] for i in range(len(basis_set))]

def v_ij(i, j, basis_set):
  integrand = sp.lambdify([[x, y, z]], V_funcs * basis_set[i] * basis_set[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  v = integrator(integrand, nitn=50, neval=10000).mean
  return v

def parallel_V_matrix(basis_set):
    n = len(basis_set)
    V = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(v_ij, i, j, basis_set) for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                V[i, j] = futures[i][j].result()
    return V

V = parallel_V_matrix(basis_set=basis_set)

np.set_printoptions(precision=12, suppress=False, linewidth=200, threshold=np.inf)
print(V)


[[-6.171632455204e+01 -7.440445115700e+00 -9.928633564927e-04 -2.500315731650e-03  8.160638151613e-03 -1.744899931448e+00 -1.744359575286e+00]
 [-7.440445115700e+00 -1.014606537052e+01  6.313484037093e-04  1.144313149818e-03  1.123473292117e-01 -3.863266438292e+00 -3.866732941065e+00]
 [-3.502181382244e-04  8.114401443605e-04 -2.497981137482e+00  2.945576190311e-04 -4.402723962542e-04 -1.417041379325e-03  7.535888023159e-06]
 [ 3.247203802945e-04 -1.739371674573e-04  1.102529374898e-03 -2.535892616199e+00 -6.254333032446e-04 -1.123535957714e+00  1.124345168026e+00]
 [ 9.978672849025e-03  1.138199991625e-01 -6.654450043801e-04  3.057193772834e-05 -2.520193737131e+00  9.122992464645e-01  9.121600005036e-01]
 [-1.745322167752e+00 -3.865230386809e+00 -1.339815347066e-04 -1.122403264225e+00  9.131073609883e-01 -5.834304506915e+00 -1.621416630355e+00]
 [-1.743563146579e+00 -3.865783222636e+00  1.564018175470e-04  1.122545974470e+00  9.115998308803e-01 -1.622520485982e+00 -5.830309282506e+00]

#3. Initial guess at the coefficient matrix C using Huckel theory *since bonding has to be specified explicitly, this is only applicable to water


In [9]:
# Obtain AO coefficient eigenvectors, then concatenate them into C
# N.B. we have to ensure that C is a 7x7 matrix so that its dimensionality...
#... matches with the Fock matrix. Therefore we assume that any pair of...
#... orbitals from different atoms = beta.

alpha, beta, E = sp.symbols('alpha beta E')

# Basis: O(1s), O(2s), O(2Px), O(2Py), O(2Pz), H_1(1s), H_2(1s)
# alpha = 0, beta = -1

Secular_Matrix = Matrix([
[- E, 0, 0, 0, 0, -1, -1],
[0, - E, 0, 0, 0, -1, -1],
[0, 0, - E, 0, 0, -1, -1],
[0, 0, 0, - E, 0, -1, -1],
[0, 0, 0, 0, - E, -1, -1],
[-1, -1, -1, -1, -1, - E, 0],
[-1, -1, -1, -1, -1, 0, - E]])

c_eigenvectors = Secular_Matrix.eigenvects()

C = []

for eigval, multiplicity, eigenvec_list in c_eigenvectors:
  for vec in eigenvec_list:
    vec = vec / vec.norm()
    C.append(vec)

C = Matrix.hstack(*C)

sp.pprint(C)

⎡-√2   -√2   -√2   -√2         √10  -√10 ⎤
⎢────  ────  ────  ────   0    ───  ─────⎥
⎢ 2     2     2     2          10    10  ⎥
⎢                                        ⎥
⎢ √2                           √10  -√10 ⎥
⎢ ──    0     0     0     0    ───  ─────⎥
⎢ 2                            10    10  ⎥
⎢                                        ⎥
⎢       √2                     √10  -√10 ⎥
⎢ 0     ──    0     0     0    ───  ─────⎥
⎢       2                      10    10  ⎥
⎢                                        ⎥
⎢             √2               √10  -√10 ⎥
⎢ 0     0     ──    0     0    ───  ─────⎥
⎢             2                10    10  ⎥
⎢                                        ⎥
⎢                   √2         √10  -√10 ⎥
⎢ 0     0     0     ──    0    ───  ─────⎥
⎢                   2          10    10  ⎥
⎢                                        ⎥
⎢                        -√2             ⎥
⎢ 0     0     0     0    ────  1/2   1/2 ⎥
⎢                         2              ⎥
⎢          

#4. Calculating S, U and X such that
#(U^T S U) = s;
#(X^T S X) = I.

In [10]:
# Calculating the overlap integral S
# Write nested function so that it can be reused in the SCF procudure later

def s_ij(i, j, basis_set):
  integrand = sp.lambdify([[x, y, z]], basis_set[i] * basis_set[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  s = integrator(integrand, nitn=50, neval=10000).mean
  return s

# Compute S elements in parallel
def parallel_S_matrix(basis_set):
    n = len(basis_set)
    S = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(s_ij, i, j, basis_set)
                    for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                S[i, j] = futures[i][j].result()
    return S

S = parallel_S_matrix(basis_set)

# Obtaining U and X
eigenvalues, eigenvectors = np.linalg.eig(S)
U = eigenvectors
s_inv_sqrt = np.diag(eigenvalues**-0.5)
X = U @ s_inv_sqrt

S = parallel_S_matrix(basis_set)

sp.pprint(S)

# Check that X^T S X = I
sp.pprint(X.T @ S @ X)


[[ 9.998362463177e-01  2.366100311691e-01  4.048709959186e-05 -3.620282357337e ↪
 [ 2.366100311691e-01  9.999057872378e-01  3.247199768660e-05 -7.089348443459e ↪
 [ 1.866274237118e-05  2.089416209914e-04  2.500081127857e-01 -3.566817394404e ↪
 [ 3.046994939251e-05  1.010474843102e-04 -1.402184827982e-04  2.499181941155e ↪
 [ 3.370059681474e-05  1.127733570352e-04  1.920393432029e-05  3.330523919984e ↪
 [ 5.388715541286e-02  4.743281345826e-01 -5.569357416442e-05  1.550935886764e ↪
 [ 5.389281245622e-02  4.743981149365e-01  4.252839151653e-05 -1.551434489930e ↪

↪ -05  3.354247807878e-05  5.388450286390e-02  5.388518496434e-02] 
↪ -05 -2.884089967817e-05  4.742651660307e-01  4.743789651117e-01] 
↪ -05  3.937862129907e-05  1.142551122886e-04  1.334947073830e-04] 
↪ -01 -2.370317288455e-05  1.551139377420e-01 -1.550380293831e-01] 
↪ -05  2.498035370577e-01 -1.207044921929e-01 -1.208877038522e-01] 
↪ -01 -1.209152407411e-01  9.995455591046e-01  2.527303424819e-01] 
↪ -01 -1.207791395014e-0

#5. Computing the density matrix P and then coulomb and exchange integrals J and K (takes ~5hrs on Google Colab, see next section to circumvent this section)

In [11]:
#Obtaining P = 2 * Simga(occupied MOs)(C_ia^* * C_ja)

C = [
    [-np.sqrt(2)/2, -np.sqrt(2)/2, -np.sqrt(2)/2, -np.sqrt(2)/2,        0,  np.sqrt(10)/10, -np.sqrt(10)/10],
    [ np.sqrt(2)/2,        0,              0,              0,          0,  np.sqrt(10)/10, -np.sqrt(10)/10],
    [       0,       np.sqrt(2)/2,         0,              0,          0,  np.sqrt(10)/10, -np.sqrt(10)/10],
    [       0,             0,        np.sqrt(2)/2,         0,          0,  np.sqrt(10)/10, -np.sqrt(10)/10],
    [       0,             0,              0,        np.sqrt(2)/2,     0,  np.sqrt(10)/10, -np.sqrt(10)/10],
    [       0,             0,              0,              0,  -np.sqrt(2)/2,     0.5,       0.5],
    [       0,             0,              0,              0,   np.sqrt(2)/2,     0.5,       0.5]
]

n = len(basis_set)
P = np.zeros((n, n))
n_occ = n_electrons // 2

C = np.array(C)
C_occ = C[:, :n_occ]


for i in range(n):
  for mu in range(n):
    for nu in range(n):
      P[mu, nu] += 2*(C[mu, i] * C[nu, i])

sp.pprint(P)

[[ 4.40000000000e+00 -6.00000000000e-01 -6.00000000000e-01 -6.00000000000e-01  ↪
 [-6.00000000000e-01  1.40000000000e+00  4.00000000000e-01  4.00000000000e-01  ↪
 [-6.00000000000e-01  4.00000000000e-01  1.40000000000e+00  4.00000000000e-01  ↪
 [-6.00000000000e-01  4.00000000000e-01  4.00000000000e-01  1.40000000000e+00  ↪
 [-6.00000000000e-01  4.00000000000e-01  4.00000000000e-01  4.00000000000e-01  ↪
 [ 0.00000000000e+00  0.00000000000e+00  0.00000000000e+00  0.00000000000e+00  ↪
 [ 0.00000000000e+00  0.00000000000e+00  0.00000000000e+00  0.00000000000e+00  ↪

↪ -6.00000000000e-01  0.00000000000e+00  0.00000000000e+00] 
↪  4.00000000000e-01  0.00000000000e+00  0.00000000000e+00] 
↪  4.00000000000e-01  0.00000000000e+00  0.00000000000e+00] 
↪  4.00000000000e-01  0.00000000000e+00  0.00000000000e+00] 
↪  1.40000000000e+00  0.00000000000e+00  0.00000000000e+00] 
↪  0.00000000000e+00  2.00000000000e+00 -2.22044604925e-16] 
↪  0.00000000000e+00 -2.22044604925e-16  2.00000000000e+00]]


In [ ]:
x1, y1, z1, x2, y2, z2 = sp.symbols('x1 y1 z1 x2 y2 z2')

# J = SIGMA(lambda sigma)(P_{lambda sigma} * (mu nu | lambda sigma))
# First define the 4-index integral (mu nu | lambda sigma):

def r12_inv(x1, y1, z1, x2, y2, z2):
    return (1 / np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))

numeric_J_integrand = np.empty((len(basis_set),)*4, dtype=object)
numeric_K_integrand = np.empty((len(basis_set),)*4, dtype=object)

def integrand_JK(x1, y1, z1, x2, y2, z2, bf1, bf2, bf3, bf4):
    r12 = sp.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
    return bf1.subs({x: x1, y: y1, z: z1}) * bf2.subs({x: x1, y: y1, z: z1}) * (1 / r12) * bf3.subs({x: x2, y: y2, z: z2}) * bf4.subs({x: x2, y: y2, z: z2})

def integrand_JK_symbolic(bf1, bf2, bf3, bf4):
    return bf1 * bf2 * (1 / sp.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)) * bf3 * bf4

numeric_J_integrand = np.empty((len(basis_set),)*4, dtype=object)
numeric_K_integrand = np.empty((len(basis_set),)*4, dtype=object)

for i in range(len(basis_set)):
    for j in range(len(basis_set)):
        for k in range(len(basis_set)):
            for l in range(len(basis_set)):
                symbolic_J_integrand = integrand_JK_symbolic(
                    basis_set[i].subs({x: x1, y: y1, z: z1}),
                    basis_set[j].subs({x: x1, y: y1, z: z1}),
                    basis_set[k].subs({x: x2, y: y2, z: z2}),
                    basis_set[l].subs({x: x2, y: y2, z: z2}),
                )
                numeric_J_integrand[i,j,k,l] = sp.lambdify(((x1, y1, z1, x2, y2, z2),), symbolic_J_integrand, 'numpy')


                symbolic_K_integrand = integrand_JK_symbolic(
                    basis_set[i].subs({x: x1, y: y1, z: z1}),
                    basis_set[k].subs({x: x1, y: y1, z: z1}),
                    basis_set[j].subs({x: x2, y: y2, z: z2}),
                    basis_set[l].subs({x: x2, y: y2, z: z2}),
                )
                numeric_K_integrand[i,j,k,l] = sp.lambdify(((x1, y1, z1, x2, y2, z2),), symbolic_K_integrand, 'numpy')


# Use vegas monte carlo integrator (faster than analytic integrators)
# Re-initialise the integrator on each run so that its monte carlo adaptive sampling doesnt carry over



def j_ij(i, j):
  j_term = 0
  integrator = vegas.Integrator([[-3,3], [-3,3], [-3,3], [-3,3], [-3,3], [-3,3]])
  for k in range(len(basis_set)):
    for l in range(len(basis_set)):
      j_term += P[k][l] * integrator(numeric_J_integrand[i,j,k,l], nitn=10, neval=10000).mean
  return j_term


def k_ij(i, j):
  k_term = 0
  integrator = vegas.Integrator([[-3,3], [-3,3], [-3,3], [-3,3], [-3,3], [-3,3]])
  for k in range(len(basis_set)):
    for l in range(len(basis_set)):
      k_term += P[k][l] * integrator(numeric_K_integrand[i,k,j,l], nitn=10, neval=10000).mean
  return k_term


# P is just a coefficient matrix so we can multiply it by the integrands pre-integration.

def parallel_J_matrix(basis_set):
    n = len(basis_set)
    J = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(j_ij, i, j) for i in range(n)] for j in range(n)]
        for i in range(n):
            for j in range(n):
                J[i, j] = futures[i][j].result()
    return J

def parallel_K_matrix(basis_set):
    n = len(basis_set)
    K = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(k_ij, i, j) for i in range(n)] for j in range(n)]
        for i in range(n):
            for j in range(n):
                K[i, j] = futures[i][j].result()
    return K

J = parallel_J_matrix(basis_set)
K = parallel_K_matrix(basis_set)

sp.pprint(J)
sp.pprint(K)

F = np.zeros((n, n))
F[i, j] = T[i, j] + V[i, j] + J[i, j] - 0.5 * K[i, j]

sp.pprint(F)

SyntaxError: expected '(' (<ipython-input-1-bb120bf64f07>, line 7)

# See below: Electron Repulsion Integral matrix (7x7x7x7 matrix) calculated using an external computer (runpod H100 PCLe).

In [3]:
eri = [[[[4.42048492e+00, 7.35376084e-01, -2.27230393e-03, -6.52506144e-05, 4.85870448e-04, 1.62599008e-01, 4.00351158e-06],
  [7.35376084e-01, 1.12387894e+00, -8.63064051e-04, -2.80186925e-03, 1.08825658e-04, 4.03556902e-01, 4.01806338e-01],
  [-2.27230393e-03, -8.63064051e-04, 2.77360633e-01, -7.25907856e-04, -2.35842771e-03, -7.50149166e-04, 1.29827165e-03],
  [-6.52506144e-05, -2.80186925e-03, -7.25907856e-04, 2.74866670e-01, 4.14071470e-04, 1.11495800e-01, -1.10880675e-01],
  [4.85870448e-04, 1.08825658e-04, -2.35842771e-03, 4.14071470e-04, 2.76023833e-01, -8.61297555e-02, -8.64325890e-02],
  [1.62599008e-01, 4.03556902e-01, -7.50149166e-04, 1.11495800e-01, -8.61297555e-02, 5.05580893e-01, 1.57539641e-01],
  [4.00351158e-06, 4.01806338e-01, 1.29827165e-03, -1.10880675e-01, -8.64325890e-02, 1.57539641e-01, 4.98928733e-01]],

 [[7.35376084e-01, 1.37250375e-01, 4.37137366e-04, 2.94325131e-04, 3.12330658e-04, 1.12105500e-02, 3.10968983e-02],
  [1.37250375e-01, 2.56912347e-01, -6.03920761e-04, -2.05935875e-04, -5.10638394e-04, 9.34315568e-02, 9.26345761e-02],
  [4.37137366e-04, -6.03920761e-04, 6.43056335e-02, 2.66728300e-04, 2.27122198e-04, 3.40569537e-04, -2.32825687e-04],
  [2.94325131e-04, -2.05935875e-04, 2.66728300e-04, 6.39931284e-02, 3.71806093e-05, 2.61847750e-02, -2.59488203e-02],
  [3.12330658e-04, -5.10638394e-04, 2.27122198e-04, 3.71806093e-05, 6.34079525e-02, -2.01772593e-02, -2.05295508e-02],
  [1.12105500e-02, 9.34315568e-02, 3.40569537e-04, 2.61847750e-02, -2.01772593e-02, 1.22062781e-01, 3.71299836e-02],
  [3.10968983e-02, 9.26345761e-02, -2.32825687e-04, -2.59488203e-02, -2.05295508e-02, 3.71299836e-02, 1.21837868e-01]],

 [[-2.27230393e-03, 4.37137366e-04, 4.83705473e-03, 3.07534869e-05, -4.11775341e-06, -2.79673624e-06, 6.49858828e-07],
  [4.37137366e-04, -1.70413193e-04, 9.25346102e-03, 3.24775538e-04, 7.38096611e-04, 2.11755748e-04, 4.29247978e-05],
  [4.83705473e-03, 9.25346102e-03, -1.77493004e-04, 3.97970610e-05, 1.43106614e-04, 2.42887765e-03, 2.46156102e-03],
  [3.07534869e-05, 3.24775538e-04, 3.97970610e-05, -9.33453956e-05, -3.96039814e-06, -1.33872490e-04, -3.60941579e-05],
  [-4.11775341e-06, 7.38096611e-04, 1.43106614e-04, -3.96039814e-06, -1.23646317e-04, -2.17873296e-06, 1.35065949e-04],
  [-2.79673624e-06, 2.11755748e-04, 2.42887765e-03, -1.33872490e-04, -2.17873296e-06, -1.71585529e-04, -6.75780472e-05],
  [6.49858828e-07, 4.29247978e-05, 2.46156102e-03, -3.60941579e-05, 1.35065949e-04, -6.75780472e-05, 2.59899668e-04]],

 [[-6.52506144e-05, 2.94325131e-04, 3.07534869e-05, 5.62289865e-05, 1.44662253e-04, 6.97531989e-04, -7.18589381e-04],
  [2.94325131e-04, 8.38715695e-05, -6.08335847e-04, 9.12858469e-03, 2.08578709e-04, 2.92987344e-03, -2.83653712e-03],
  [3.07534869e-05, -6.08335847e-04, 9.91814895e-05, 3.07205309e-05, 4.71592999e-05, -7.90234080e-05, -9.85978867e-05],
  [5.62289865e-05, 9.12858469e-03, 3.07205309e-05, 1.66615258e-04, 4.37747571e-05, 3.07287008e-03, 3.26859611e-03],
  [1.44662253e-04, 2.08578709e-04, 4.71592999e-05, 4.37747571e-05, -1.62589865e-04, -5.18257778e-04, 6.06447456e-04],
  [6.97531989e-04, 2.92987344e-03, -7.90234080e-05, 3.07287008e-03, -5.18257778e-04, 4.73895091e-03, -6.16448449e-05],
  [-7.18589381e-04, -2.83653712e-03, -9.85978867e-05, 3.26859611e-03, 6.06447456e-04, -6.16448449e-05, -4.70502623e-03]],

 [[4.85870448e-04, 3.12330658e-04, -4.11775341e-06, 1.44662253e-04, 6.90931607e-04, -7.04368738e-04, -4.10408453e-04],
  [3.12330658e-04, 1.23481002e-04, -6.32179929e-05, 2.60487188e-04, 9.61988122e-03, -1.76830573e-03, -2.34600639e-03],
  [-4.11775341e-06, -6.32179929e-05, 1.93044954e-04, 3.66028286e-05, 1.60263199e-04, -1.49184716e-04, 1.45186671e-05],
  [1.44662253e-04, 2.60487188e-04, 3.66028286e-05, -1.88411555e-05, 1.82330197e-04, -5.74151302e-04, 5.36341249e-04],
  [6.90931607e-04, 9.61988122e-03, 1.60263199e-04, 1.82330197e-04, -2.32142950e-04, 2.81761725e-03, 2.72220434e-03],
  [-7.04368738e-04, -1.76830573e-03, -1.49184716e-04, -5.74151302e-04, 2.81761725e-03, -4.53620477e-03, -1.19177574e-03],
  [-4.10408453e-04, -2.34600639e-03, 1.45186671e-05, 5.36341249e-04, 2.72220434e-03, -1.19177574e-03, -3.65473452e-03]],

 [[1.62599008e-01, 1.12105500e-02, -2.79673624e-06, 6.97531989e-04, -7.04368738e-04, 5.01967630e-05, 8.83333973e-04],
  [1.12105500e-02, 5.86693917e-02, -2.94591708e-04, 1.01951371e-03, -8.21261404e-04, 2.16905230e-02, 2.11371107e-02],
  [-2.79673624e-06, -2.94591708e-04, 1.44988464e-02, 6.46947496e-05, -6.26784906e-05, -2.61486504e-05, -7.35524585e-05],
  [6.97531989e-04, 1.01951371e-03, 6.46947496e-05, 1.45087767e-02, -7.83038941e-05, 6.43138835e-03, -5.59544960e-03],
  [-7.04368738e-04, -8.21261404e-04, -6.26784906e-05, -7.83038941e-05, 1.46173010e-02, -4.96801944e-03, -4.84755835e-03],
  [5.01967630e-05, 2.16905230e-02, -2.61486504e-05, 6.43138835e-03, -4.96801944e-03, 2.91791368e-02, 8.62274816e-03],
  [8.83333973e-04, 2.11371107e-02, -7.35524585e-05, -5.59544960e-03, -4.84755835e-03, 8.62274816e-03, 2.73423661e-02]],

 [[4.00351158e-06, 3.10968983e-02, 6.49858828e-07, -7.18589381e-04, -4.10408453e-04, 8.83333973e-04, 7.23376172e-03],
  [3.10968983e-02, 5.76087094e-02, 1.47309077e-04, -9.57952552e-04, -8.83686910e-04, 2.11836266e-02, 2.18692780e-02],
  [6.49858828e-07, 1.47309077e-04, 1.45160040e-02, -4.67336341e-05, 7.43045609e-05, -3.93922391e-05, -4.29018115e-05],
  [-7.18589381e-04, -9.57952552e-04, -4.67336341e-05, 1.45732653e-02, 4.42423492e-06, 5.69780264e-03, -6.41620131e-03],
  [-4.10408453e-04, -8.83686910e-04, 7.43045609e-05, 4.42423492e-06, 1.46622362e-02, -4.86348005e-03, -4.98734720e-03],
  [8.83333973e-04, 2.11836266e-02, -3.93922391e-05, 5.69780264e-03, -4.86348005e-03, 2.75648781e-02, 8.58054677e-03],
  [7.23376172e-03, 2.18692780e-02, -4.29018115e-05, -6.41620131e-03, -4.98734720e-03, 8.58054677e-03, 2.90661444e-02]],

 [[7.35376084e-01, 1.37250375e-01, 4.37137366e-04, 2.94325131e-04, 3.12330658e-04, 1.12105500e-02, 3.10968983e-02],
  [1.37250375e-01, 2.56912347e-01, -6.03920761e-04, -2.05935875e-04, -5.10638394e-04, 9.34315568e-02, 9.26345761e-02],
  [4.37137366e-04, -6.03920761e-04, 6.43056335e-02, 2.66728300e-04, 2.27122198e-04, 3.40569537e-04, -2.32825687e-04],
  [2.94325131e-04, -2.05935875e-04, 2.66728300e-04, 6.39931284e-02, 3.71806093e-05, 2.61847750e-02, -2.59488203e-02],
  [3.12330658e-04, -5.10638394e-04, 2.27122198e-04, 3.71806093e-05, 6.34079525e-02, -2.01772593e-02, -2.05295508e-02],
  [1.12105500e-02, 9.34315568e-02, 3.40569537e-04, 2.61847750e-02, -2.01772593e-02, 1.22062781e-01, 3.71299836e-02],
  [3.10968983e-02, 9.26345761e-02, -2.32825687e-04, -2.59488203e-02, -2.05295508e-02, 3.71299836e-02, 1.21837868e-01]],

 [[1.12387894e+00, 2.56912347e-01, -1.70413193e-04, 8.38715695e-05, 1.23481002e-04, 5.86693917e-02, 5.76087094e-02],
  [2.56912347e-01, 8.22099770e-01, 5.29515156e-04, -6.69397059e-04, -3.04792293e-03, 3.28169330e-01, 3.27331193e-01],
  [-1.70413193e-04, 5.29515156e-04, 2.02343832e-01, -4.49123043e-04, 5.70063454e-04, -1.16540139e-03, -1.07926173e-03],
  [8.38715695e-05, -6.69397059e-04, -4.49123043e-04, 2.04082405e-01, 8.56299165e-04, 9.84277485e-02, -9.96812292e-02],
  [1.23481002e-04, -3.04792293e-03, 5.70063454e-04, 8.56299165e-04, 2.03759837e-01, -7.63569162e-02, -7.71586463e-02],
  [5.86693917e-02, 3.28169330e-01, -1.16540139e-03, 9.84277485e-02, -7.63569162e-02, 4.89486454e-01, 1.41292648e-01],
  [5.76087094e-02, 3.27331193e-01, -1.07926173e-03, -9.96812292e-02, -7.71586463e-02, 1.41292648e-01, 4.89352274e-01]],

 [[-8.63064051e-04, -6.03920761e-04, 9.25346102e-03, -6.08335847e-04, -6.32179929e-05, -2.94591708e-04, 1.47309077e-04],
  [-6.03920761e-04, 5.29515156e-04, 4.56998813e-02, -8.40894250e-04, 8.81225927e-05, 1.41632573e-03, 1.11908108e-03],
  [9.25346102e-03, 4.56998813e-02, -3.07746456e-04, 2.59535962e-04, -8.06263340e-05, 1.46794584e-02, 1.40652329e-02],
  [-6.08335847e-04, -8.40894250e-04, 2.59535962e-04, 1.01380051e-03, 1.06191075e-04, 6.19213925e-04, 6.39578084e-04],
  [-6.32179929e-05, 8.81225927e-05, -8.06263340e-05, 1.06191075e-04, -8.42075037e-04, 5.33726299e-04, 1.49986246e-04],
  [-2.94591708e-04, 1.41632573e-03, 1.46794584e-02, 6.19213925e-04, 5.33726299e-04, 5.48978733e-05, -8.09036149e-05],
  [1.47309077e-04, 1.11908108e-03, 1.40652329e-02, 6.39578084e-04, 1.49986246e-04, -8.09036149e-05, -4.05768050e-04]],

 [[-2.80186925e-03, -2.05935875e-04, 3.24775538e-04, 9.12858469e-03, 2.60487188e-04, 1.01951371e-03, -9.57952552e-04],
  [-2.05935875e-04, -6.69397059e-04, -8.40894250e-04, 4.53197855e-02, 1.70967855e-03, 2.11500099e-02, -2.08755834e-02],
  [3.24775538e-04, -8.40894250e-04, -4.18621596e-04, 4.75564602e-04, -7.87096767e-05, -1.55727753e-04, 2.09686835e-04],
  [9.12858469e-03, 4.53197855e-02, 4.75564602e-04, 1.12162890e-04, -5.44750314e-04, 2.02222767e-02, 1.91463504e-02],
  [2.60487188e-04, 1.70967855e-03, -7.87096767e-05, -5.44750314e-04, 5.33274238e-04, -4.73588291e-03, 4.96602923e-03],
  [1.01951371e-03, 2.11500099e-02, -1.55727753e-04, 2.02222767e-02, -4.73588291e-03, 4.98078965e-02, -2.17599478e-05],
  [-9.57952552e-04, -2.08755834e-02, 2.09686835e-04, 1.91463504e-02, 4.96602923e-03, -2.17599478e-05, -4.93590776e-02]],

 [[1.08825658e-04, -5.10638394e-04, 7.38096611e-04, 2.08578709e-04, 9.61988122e-03, -8.21261404e-04, -8.83686910e-04],
  [-5.10638394e-04, -3.04792293e-03, 8.81225927e-05, 1.70967855e-03, 4.31783543e-02, -1.66527055e-02, -1.51018894e-02],
  [7.38096611e-04, 8.81225927e-05, 8.81991988e-04, 2.61150512e-04, 1.95721515e-04, -7.23893422e-05, 1.49611879e-04],
  [2.08578709e-04, 1.70967855e-03, 2.61150512e-04, -6.00827895e-04, 1.98679691e-04, -4.44643489e-03, 4.46299059e-03],
  [9.61988122e-03, 4.31783543e-02, 1.95721515e-04, 1.98679691e-04, 1.23986184e-03, 1.75433921e-02, 1.78667560e-02],
  [-8.21261404e-04, -1.66527055e-02, -7.23893422e-05, -4.44643489e-03, 1.75433921e-02, -4.00257770e-02, -1.21664983e-02],
  [-8.83686910e-04, -1.51018894e-02, 1.49611879e-04, 4.46299059e-03, 1.78667560e-02, -1.21664983e-02, -3.95184202e-02]],

 [[4.03556902e-01, 9.34315568e-02, 2.11755748e-04, 2.92987344e-03, -1.76830573e-03, 2.16905230e-02, 2.11836266e-02],
  [9.34315568e-02, 3.28169330e-01, 1.41632573e-03, 2.11500099e-02, -1.66527055e-02, 1.59733404e-01, 1.31352053e-01],
  [2.11755748e-04, 1.41632573e-03, 8.06030855e-02, -4.27228551e-04, 5.72070762e-04, -9.43811034e-06, 3.23194627e-04],
  [2.92987344e-03, 2.11500099e-02, -4.27228551e-04, 8.32311547e-02, -1.40320465e-03, 5.81120153e-02, -3.31469356e-02],
  [-1.76830573e-03, -1.66527055e-02, 5.72070762e-04, -1.40320465e-03, 8.16714345e-02, -4.62068609e-02, -3.66410274e-02],
  [2.16905230e-02, 1.59733404e-01, -9.43811034e-06, 5.81120153e-02, -4.62068609e-02, 2.97462610e-01, 6.77195435e-02],
  [2.11836266e-02, 1.31352053e-01, 3.23194627e-04, -3.31469356e-02, -3.66410274e-02, 6.77195435e-02, 2.01600367e-01]],

 [[4.01806338e-01, 9.26345761e-02, 4.29247978e-05, -2.83653712e-03, -2.34600639e-03, 2.11371107e-02, 2.18692780e-02],
  [9.26345761e-02, 3.27331193e-01, 1.11908108e-03, -2.08755834e-02, -1.51018894e-02, 1.31352053e-01, 1.59988542e-01],
  [4.29247978e-05, 1.11908108e-03, 8.00035122e-02, -3.29573637e-04, 5.42159273e-04, -1.36642685e-04, -4.39298987e-04],
  [-2.83653712e-03, -2.08755834e-02, -3.29573637e-04, 8.30857196e-02, 1.87375871e-03, 3.29099781e-02, -5.82894086e-02],
  [-2.34600639e-03, -1.51018894e-02, 5.42159273e-04, 1.87375871e-03, 8.14704829e-02, -3.66558200e-02, -4.56411322e-02],
  [2.11371107e-02, 1.31352053e-01, -1.36642685e-04, 3.29099781e-02, -3.66558200e-02, 2.01950063e-01, 6.79235993e-02],
  [2.18692780e-02, 1.59988542e-01, -4.39298987e-04, -5.82894086e-02, -4.56411322e-02, 6.79235993e-02, 2.99258159e-01]],

 [[-2.27230393e-03, 4.37137366e-04, 4.83705473e-03, 3.07534869e-05, -4.11775341e-06, -2.79673624e-06, 6.49858828e-07],
  [4.37137366e-04, -1.70413193e-04, 9.25346102e-03, 3.24775538e-04, 7.38096611e-04, 2.11755748e-04, 4.29247978e-05],
  [4.83705473e-03, 9.25346102e-03, -1.77493004e-04, 3.97970610e-05, 1.43106614e-04, 2.42887765e-03, 2.46156102e-03],
  [3.07534869e-05, 3.24775538e-04, 3.97970610e-05, -9.33453956e-05, -3.96039814e-06, -1.33872490e-04, -3.60941579e-05],
  [-4.11775341e-06, 7.38096611e-04, 1.43106614e-04, -3.96039814e-06, -1.23646317e-04, -2.17873296e-06, 1.35065949e-04],
  [-2.79673624e-06, 2.11755748e-04, 2.42887765e-03, -1.33872490e-04, -2.17873296e-06, -1.71585529e-04, -6.75780472e-05],
  [6.49858828e-07, 4.29247978e-05, 2.46156102e-03, -3.60941579e-05, 1.35065949e-04, -6.75780472e-05, 2.59899668e-04]],

 [[-8.63064051e-04, -6.03920761e-04, 9.25346102e-03, -6.08335847e-04, -6.32179929e-05, -2.94591708e-04, 1.47309077e-04],
  [-6.03920761e-04, 5.29515156e-04, 4.56998813e-02, -8.40894250e-04, 8.81225927e-05, 1.41632573e-03, 1.11908108e-03],
  [9.25346102e-03, 4.56998813e-02, -3.07746456e-04, 2.59535962e-04, -8.06263340e-05, 1.46794584e-02, 1.40652329e-02],
  [-6.08335847e-04, -8.40894250e-04, 2.59535962e-04, 1.01380051e-03, 1.06191075e-04, 6.19213925e-04, 6.39578084e-04],
  [-6.32179929e-05, 8.81225927e-05, -8.06263340e-05, 1.06191075e-04, -8.42075037e-04, 5.33726299e-04, 1.49986246e-04],
  [-2.94591708e-04, 1.41632573e-03, 1.46794584e-02, 6.19213925e-04, 5.33726299e-04, 5.48978733e-05, -8.09036149e-05],
  [1.47309077e-04, 1.11908108e-03, 1.40652329e-02, 6.39578084e-04, 1.49986246e-04, -8.09036149e-05, -4.05768050e-04]],

 [[2.77360633e-01, 6.43056335e-02, -1.77493004e-04, 9.91814895e-05, 1.93044954e-04, 1.44988464e-02, 1.45160040e-02],
  [6.43056335e-02, 2.02343832e-01, -3.07746456e-04, -4.18621596e-04, 8.81991988e-04, 8.06030855e-02, 8.00035122e-02],
  [-1.77493004e-04, -3.07746456e-04, 5.44595779e-02, 1.55106687e-04, -3.29392663e-04, 8.56563252e-05, 2.88882745e-04],
  [9.91814895e-05, -4.18621596e-04, 1.55106687e-04, 4.86085434e-02, 5.91580312e-05, 2.37756230e-02, -2.37961138e-02],
  [1.93044954e-04, 8.81991988e-04, -3.29392663e-04, 5.91580312e-05, 4.88902401e-02, -1.84997600e-02, -1.83002481e-02],
  [1.44988464e-02, 8.06030855e-02, 8.56563252e-05, 2.37756230e-02, -1.84997600e-02, 1.17109249e-01, 3.42969014e-02],
  [1.45160040e-02, 8.00035122e-02, 2.88882745e-04, -2.37961138e-02, -1.83002481e-02, 3.42969014e-02, 1.17020219e-01]],

 [[-7.25907856e-04, 2.66728300e-04, 3.97970610e-05, 3.07205309e-05, 3.66028286e-05, 6.46947496e-05, -4.67336341e-05],
  [2.66728300e-04, -4.49123043e-04, 2.59535962e-04, 4.75564602e-04, 2.61150512e-04, -4.27228551e-04, -3.29573637e-04],
  [3.97970610e-05, 2.59535962e-04, 1.55106687e-04, 2.77330237e-03, -2.02580204e-04, 1.20900700e-03, -1.13841056e-03],
  [3.07205309e-05, 4.75564602e-04, 2.77330237e-03, -3.95199184e-04, -4.83653525e-07, 2.48754383e-04, 2.42038461e-04],
  [3.66028286e-05, 2.61150512e-04, -2.02580204e-04, -4.83653525e-07, 1.82278307e-04, 1.15699858e-04, 3.78523743e-04],
  [6.46947496e-05, -4.27228551e-04, 1.20900700e-03, 2.48754383e-04, 1.15699858e-04, -4.07503249e-04, -1.65981732e-04],
  [-4.67336341e-05, -3.29573637e-04, -1.13841056e-03, 2.42038461e-04, 3.78523743e-04, -1.65981732e-04, 2.09471598e-04]],

 [[-2.35842771e-03, 2.27122198e-04, 1.43106614e-04, 4.71592999e-05, 1.60263199e-04, -6.26784906e-05, 7.43045609e-05],
  [2.27122198e-04, 5.70063454e-04, -8.06263340e-05, -7.87096767e-05, 1.95721515e-04, 5.72070762e-04, 5.42159273e-04],
  [1.43106614e-04, -8.06263340e-05, -3.29392663e-04, -2.02580204e-04, 3.07835644e-03, -7.49673180e-04, -9.02108014e-04],
  [4.71592999e-05, -7.87096767e-05, -2.02580204e-04, -1.84068116e-04, -1.87994014e-04, 1.38811570e-04, -2.68956326e-05],
  [1.60263199e-04, 1.95721515e-04, 3.07835644e-03, -1.87994014e-04, 1.79461867e-04, -8.81288897e-06, -1.18006894e-04],
  [-6.26784906e-05, 5.72070762e-04, -7.49673180e-04, 1.38811570e-04, -8.81288897e-06, -1.66372461e-04, 2.28673607e-04],
  [7.43045609e-05, 5.42159273e-04, -9.02108014e-04, -2.68956326e-05, -1.18006894e-04, 2.28673607e-04, 1.94116251e-04]],

 [[-7.50149166e-04, 3.40569537e-04, 2.42887765e-03, -7.90234080e-05, -1.49184716e-04, -2.61486504e-05, -3.93922391e-05],
  [3.40569537e-04, -1.16540139e-03, 1.46794584e-02, -1.55727753e-04, -7.23893422e-05, -9.43811034e-06, -1.36642685e-04],
  [2.42887765e-03, 1.46794584e-02, 8.56563252e-05, 1.20900700e-03, -7.49673180e-04, 5.85980673e-03, 4.54050335e-03],
  [-7.90234080e-05, -1.55727753e-04, 1.20900700e-03, 1.46127019e-04, -8.86596671e-06, 3.02343011e-04, 4.92826524e-05],
  [-1.49184716e-04, -7.23893422e-05, -7.49673180e-04, -8.86596671e-06, 1.20154775e-04, -2.53031026e-04, -4.86193264e-07],
  [-2.61486504e-05, -9.43811034e-06, 5.85980673e-03, 3.02343011e-04, -2.53031026e-04, -1.07368417e-03, 1.06840907e-05],
  [-3.93922391e-05, -1.36642685e-04, 4.54050335e-03, 4.92826524e-05, -4.86193264e-07, 1.06840907e-05, 2.32412353e-04]],

 [[1.29827165e-03, -2.32825687e-04, 2.46156102e-03, -9.85978867e-05, 1.45186671e-05, -7.35524585e-05, -4.29018115e-05],
  [-2.32825687e-04, -1.07926173e-03, 1.40652329e-02, 2.09686835e-04, 1.49611879e-04, 3.23194627e-04, -4.39298987e-04],
  [2.46156102e-03, 1.40652329e-02, 2.88882745e-04, -1.13841056e-03, -9.02108014e-04, 4.54050335e-03, 5.82326513e-03],
  [-9.85978867e-05, 2.09686835e-04, -1.13841056e-03, 1.19490330e-04, -6.22892795e-05, 3.33545782e-05, -4.21000435e-05],
  [1.45186671e-05, 1.49611879e-04, -9.02108014e-04, -6.22892795e-05, 1.89398340e-04, -2.49235063e-05, -1.30096152e-04],
  [-7.35524585e-05, 3.23194627e-04, 4.54050335e-03, 3.33545782e-05, -2.49235063e-05, -1.35019138e-04, 1.63705951e-04],
  [-4.29018115e-05, -4.39298987e-04, 5.82326513e-03, -4.21000435e-05, -1.30096152e-04, 1.63705951e-04, -2.31729058e-04]],

 [[-6.52506144e-05, 2.94325131e-04, 3.07534869e-05, 5.62289865e-05, 1.44662253e-04, 6.97531989e-04, -7.18589381e-04],
  [2.94325131e-04, 8.38715695e-05, -6.08335847e-04, 9.12858469e-03, 2.08578709e-04, 2.92987344e-03, -2.83653712e-03],
  [3.07534869e-05, -6.08335847e-04, 9.91814895e-05, 3.07205309e-05, 4.71592999e-05, -7.90234080e-05, -9.85978867e-05],
  [5.62289865e-05, 9.12858469e-03, 3.07205309e-05, 1.66615258e-04, 4.37747571e-05, 3.07287008e-03, 3.26859611e-03],
  [1.44662253e-04, 2.08578709e-04, 4.71592999e-05, 4.37747571e-05, -1.62589865e-04, -5.18257778e-04, 6.06447456e-04],
  [6.97531989e-04, 2.92987344e-03, -7.90234080e-05, 3.07287008e-03, -5.18257778e-04, 4.73895091e-03, -6.16448449e-05],
  [-7.18589381e-04, -2.83653712e-03, -9.85978867e-05, 3.26859611e-03, 6.06447456e-04, -6.16448449e-05, -4.70502623e-03]],

 [[-2.80186925e-03, -2.05935875e-04, 3.24775538e-04, 9.12858469e-03, 2.60487188e-04, 1.01951371e-03, -9.57952552e-04],
  [-2.05935875e-04, -6.69397059e-04, -8.40894250e-04, 4.53197855e-02, 1.70967855e-03, 2.11500099e-02, -2.08755834e-02],
  [3.24775538e-04, -8.40894250e-04, -4.18621596e-04, 4.75564602e-04, -7.87096767e-05, -1.55727753e-04, 2.09686835e-04],
  [9.12858469e-03, 4.53197855e-02, 4.75564602e-04, 1.12162890e-04, -5.44750314e-04, 2.02222767e-02, 1.91463504e-02],
  [2.60487188e-04, 1.70967855e-03, -7.87096767e-05, -5.44750314e-04, 5.33274238e-04, -4.73588291e-03, 4.96602923e-03],
  [1.01951371e-03, 2.11500099e-02, -1.55727753e-04, 2.02222767e-02, -4.73588291e-03, 4.98078965e-02, -2.17599478e-05],
  [-9.57952552e-04, -2.08755834e-02, 2.09686835e-04, 1.91463504e-02, 4.96602923e-03, -2.17599478e-05, -4.93590776e-02]],

 [[-7.25907856e-04, 2.66728300e-04, 3.97970610e-05, 3.07205309e-05, 3.66028286e-05, 6.46947496e-05, -4.67336341e-05],
  [2.66728300e-04, -4.49123043e-04, 2.59535962e-04, 4.75564602e-04, 2.61150512e-04, -4.27228551e-04, -3.29573637e-04],
  [3.97970610e-05, 2.59535962e-04, 1.55106687e-04, 2.77330237e-03, -2.02580204e-04, 1.20900700e-03, -1.13841056e-03],
  [3.07205309e-05, 4.75564602e-04, 2.77330237e-03, -3.95199184e-04, -4.83653525e-07, 2.48754383e-04, 2.42038461e-04],
  [3.66028286e-05, 2.61150512e-04, -2.02580204e-04, -4.83653525e-07, 1.82278307e-04, 1.15699858e-04, 3.78523743e-04],
  [6.46947496e-05, -4.27228551e-04, 1.20900700e-03, 2.48754383e-04, 1.15699858e-04, -4.07503249e-04, -1.65981732e-04],
  [-4.67336341e-05, -3.29573637e-04, -1.13841056e-03, 2.42038461e-04, 3.78523743e-04, -1.65981732e-04, 2.09471598e-04]],

 [[2.74866670e-01, 6.39931284e-02, -9.33453956e-05, 1.66615258e-04, -1.88411555e-05, 1.45087767e-02, 1.45732653e-02],
  [6.39931284e-02, 2.04082405e-01, 1.01380051e-03, 1.12162890e-04, -6.00827895e-04, 8.32311547e-02, 8.30857196e-02],
  [-9.33453956e-05, 1.01380051e-03, 4.86085434e-02, -3.95199184e-04, -1.84068116e-04, 1.46127019e-04, 1.19490330e-04],
  [1.66615258e-04, 1.12162890e-04, -3.95199184e-04, 5.44223622e-02, -2.91329707e-04, 2.62468253e-02, -2.64198829e-02],
  [-1.88411555e-05, -6.00827895e-04, -1.84068116e-04, -2.91329707e-04, 4.90249431e-02, -1.88166766e-02, -1.90460033e-02],
  [1.45087767e-02, 8.32311547e-02, 1.46127019e-04, 2.62468253e-02, -1.88166766e-02, 1.26360985e-01, 3.51531143e-02],
  [1.45732653e-02, 8.30857196e-02, 1.19490330e-04, -2.64198829e-02, -1.90460033e-02, 3.51531143e-02, 1.27312815e-01]],

 [[4.14071470e-04, 3.71806093e-05, -3.96039814e-06, 4.37747571e-05, 1.82330197e-04, -7.83038941e-05, 4.42423492e-06],
  [3.71806093e-05, 8.56299165e-04, 1.06191075e-04, -5.44750314e-04, 1.98679691e-04, -1.40320465e-03, 1.87375871e-03],
  [-3.96039814e-06, 1.06191075e-04, 5.91580312e-05, -4.83653525e-07, -1.87994014e-04, -8.86596671e-06, -6.22892795e-05],
  [4.37747571e-05, -5.44750314e-04, -4.83653525e-07, -2.91329707e-04, 2.75726697e-03, -1.53309897e-03, -1.37844042e-03],
  [1.82330197e-04, 1.98679691e-04, -1.87994014e-04, 2.75726697e-03, 1.15000550e-04, 1.47765642e-03, -1.35535286e-03],
  [-7.83038941e-05, -1.40320465e-03, -8.86596671e-06, -1.53309897e-03, 1.47765642e-03, -7.91919447e-03, -1.35438964e-04],
  [4.42423492e-06, 1.87375871e-03, -6.22892795e-05, -1.37844042e-03, -1.35535286e-03, -1.35438964e-04, 7.45387169e-03]],

 [[1.11495800e-01, 2.61847750e-02, -1.33872490e-04, 3.07287008e-03, -5.74151302e-04, 6.43138835e-03, 5.69780264e-03],
  [2.61847750e-02, 9.84277485e-02, 6.19213925e-04, 2.02222767e-02, -4.44643489e-03, 5.81120153e-02, 3.29099781e-02],
  [-1.33872490e-04, 6.19213925e-04, 2.37756230e-02, 2.48754383e-04, 1.38811570e-04, 3.02343011e-04, 3.33545782e-05],
  [3.07287008e-03, 2.02222767e-02, 2.48754383e-04, 2.62468253e-02, -1.53309897e-03, 2.67712208e-02, -3.58473186e-03],
  [-5.74151302e-04, -4.44643489e-03, 1.38811570e-04, -1.53309897e-03, 2.38544801e-02, -1.65047298e-02, -8.84816798e-03],
  [6.43138835e-03, 5.81120153e-02, 3.02343011e-04, 2.67712208e-02, -1.65047298e-02, 1.25283792e-01, 2.16572435e-02],
  [5.69780264e-03, 3.29099781e-02, 3.33545782e-05, -3.58473186e-03, -8.84816798e-03, 2.16572435e-02, 4.38469704e-02]],

 [[-1.10880675e-01, -2.59488203e-02, -3.60941579e-05, 3.26859611e-03, 5.36341249e-04, -5.59544960e-03, -6.41620131e-03],
  [-2.59488203e-02, -9.96812292e-02, 6.39578084e-04, 1.91463504e-02, 4.46299059e-03, -3.31469356e-02, -5.82894086e-02],
  [-3.60941579e-05, 6.39578084e-04, -2.37961138e-02, 2.42038461e-04, -2.68956326e-05, 4.92826524e-05, -4.21000435e-05],
  [3.26859611e-03, 1.91463504e-02, 2.42038461e-04, -2.64198829e-02, -1.37844042e-03, -3.58473186e-03, 2.69414564e-02],
  [5.36341249e-04, 4.46299059e-03, -2.68956326e-05, -1.37844042e-03, -2.42297112e-02, 8.81473816e-03, 1.65840526e-02],
  [-5.59544960e-03, -3.31469356e-02, 4.92826524e-05, -3.58473186e-03, 8.81473816e-03, -4.44667980e-02, -2.12048276e-02],
  [-6.41620131e-03, -5.82894086e-02, -4.21000435e-05, 2.69414564e-02, 1.65840526e-02, -2.12048276e-02, -1.23474124e-01]],

 [[4.85870448e-04, 3.12330658e-04, -4.11775341e-06, 1.44662253e-04, 6.90931607e-04, -7.04368738e-04, -4.10408453e-04],
  [3.12330658e-04, 1.23481002e-04, -6.32179929e-05, 2.60487188e-04, 9.61988122e-03, -1.76830573e-03, -2.34600639e-03],
  [-4.11775341e-06, -6.32179929e-05, 1.93044954e-04, 3.66028286e-05, 1.60263199e-04, -1.49184716e-04, 1.45186671e-05],
  [1.44662253e-04, 2.60487188e-04, 3.66028286e-05, -1.88411555e-05, 1.82330197e-04, -5.74151302e-04, 5.36341249e-04],
  [6.90931607e-04, 9.61988122e-03, 1.60263199e-04, 1.82330197e-04, -2.32142950e-04, 2.81761725e-03, 2.72220434e-03],
  [-7.04368738e-04, -1.76830573e-03, -1.49184716e-04, -5.74151302e-04, 2.81761725e-03, -4.53620477e-03, -1.19177574e-03],
  [-4.10408453e-04, -2.34600639e-03, 1.45186671e-05, 5.36341249e-04, 2.72220434e-03, -1.19177574e-03, -3.65473452e-03]],

 [[1.08825658e-04, -5.10638394e-04, 7.38096611e-04, 2.08578709e-04, 9.61988122e-03, -8.21261404e-04, -8.83686910e-04],
  [-5.10638394e-04, -3.04792293e-03, 8.81225927e-05, 1.70967855e-03, 4.31783543e-02, -1.66527055e-02, -1.51018894e-02],
  [7.38096611e-04, 8.81225927e-05, 8.81991988e-04, 2.61150512e-04, 1.95721515e-04, -7.23893422e-05, 1.49611879e-04],
  [2.08578709e-04, 1.70967855e-03, 2.61150512e-04, -6.00827895e-04, 1.98679691e-04, -4.44643489e-03, 4.46299059e-03],
  [9.61988122e-03, 4.31783543e-02, 1.95721515e-04, 1.98679691e-04, 1.23986184e-03, 1.75433921e-02, 1.78667560e-02],
  [-8.21261404e-04, -1.66527055e-02, -7.23893422e-05, -4.44643489e-03, 1.75433921e-02, -4.00257770e-02, -1.21664983e-02],
  [-8.83686910e-04, -1.51018894e-02, 1.49611879e-04, 4.46299059e-03, 1.78667560e-02, -1.21664983e-02, -3.95184202e-02]],

 [[-2.35842771e-03, 2.27122198e-04, 1.43106614e-04, 4.71592999e-05, 1.60263199e-04, -6.26784906e-05, 7.43045609e-05],
  [2.27122198e-04, 5.70063454e-04, -8.06263340e-05, -7.87096767e-05, 1.95721515e-04, 5.72070762e-04, 5.42159273e-04],
  [1.43106614e-04, -8.06263340e-05, -3.29392663e-04, -2.02580204e-04, 3.07835644e-03, -7.49673180e-04, -9.02108014e-04],
  [4.71592999e-05, -7.87096767e-05, -2.02580204e-04, -1.84068116e-04, -1.87994014e-04, 1.38811570e-04, -2.68956326e-05],
  [1.60263199e-04, 1.95721515e-04, 3.07835644e-03, -1.87994014e-04, 1.79461867e-04, -8.81288897e-06, -1.18006894e-04],
  [-6.26784906e-05, 5.72070762e-04, -7.49673180e-04, 1.38811570e-04, -8.81288897e-06, -1.66372461e-04, 2.28673607e-04],
  [7.43045609e-05, 5.42159273e-04, -9.02108014e-04, -2.68956326e-05, -1.18006894e-04, 2.28673607e-04, 1.94116251e-04]],

 [[4.14071470e-04, 3.71806093e-05, -3.96039814e-06, 4.37747571e-05, 1.82330197e-04, -7.83038941e-05, 4.42423492e-06],
  [3.71806093e-05, 8.56299165e-04, 1.06191075e-04, -5.44750314e-04, 1.98679691e-04, -1.40320465e-03, 1.87375871e-03],
  [-3.96039814e-06, 1.06191075e-04, 5.91580312e-05, -4.83653525e-07, -1.87994014e-04, -8.86596671e-06, -6.22892795e-05],
  [4.37747571e-05, -5.44750314e-04, -4.83653525e-07, -2.91329707e-04, 2.75726697e-03, -1.53309897e-03, -1.37844042e-03],
  [1.82330197e-04, 1.98679691e-04, -1.87994014e-04, 2.75726697e-03, 1.15000550e-04, 1.47765642e-03, -1.35535286e-03],
  [-7.83038941e-05, -1.40320465e-03, -8.86596671e-06, -1.53309897e-03, 1.47765642e-03, -7.91919447e-03, -1.35438964e-04],
  [4.42423492e-06, 1.87375871e-03, -6.22892795e-05, -1.37844042e-03, -1.35535286e-03, -1.35438964e-04, 7.45387169e-03]],

[[1.11495800e-01, 2.61847750e-02, -1.33872490e-04, 3.07287008e-03, -5.74151302e-04, 6.43138835e-03, 5.69780264e-03],
  [2.61847750e-02, 9.84277485e-02, 6.19213925e-04, 2.02222767e-02, -4.44643489e-03, 5.81120153e-02, 3.29099781e-02],
  [-1.33872490e-04, 6.19213925e-04, 2.37756230e-02, 2.48754383e-04, 1.38811570e-04, 3.02343011e-04, 3.33545782e-05],
  [3.07287008e-03, 2.02222767e-02, 2.48754383e-04, 2.62468253e-02, -1.53309897e-03, 2.67712208e-02, -3.58473186e-03],
  [-5.74151302e-04, -4.44643489e-03, 1.38811570e-04, -1.53309897e-03, 2.38544801e-02, -1.65047298e-02, -8.84816798e-03],
  [6.43138835e-03, 5.81120153e-02, 3.02343011e-04, 2.67712208e-02, -1.65047298e-02, 1.25283792e-01, 2.16572435e-02],
  [5.69780264e-03, 3.29099781e-02, 3.33545782e-05, -3.58473186e-03, -8.84816798e-03, 2.16572435e-02, 4.38469704e-02]],

 [[-1.10880675e-01, -2.59488203e-02, -3.60941579e-05, 3.26859611e-03, 5.36341249e-04, -5.59544960e-03, -6.41620131e-03],
  [-2.59488203e-02, -9.96812292e-02, 6.39578084e-04, 1.91463504e-02, 4.46299059e-03, -3.31469356e-02, -5.82894086e-02],
  [-3.60941579e-05, 6.39578084e-04, -2.37961138e-02, 2.42038461e-04, -2.68956326e-05, 4.92826524e-05, -4.21000435e-05],
  [3.26859611e-03, 1.91463504e-02, 2.42038461e-04, -2.64198829e-02, -1.37844042e-03, -3.58473186e-03, 2.69414564e-02],
  [5.36341249e-04, 4.46299059e-03, -2.68956326e-05, -1.37844042e-03, -2.42297112e-02, 8.81473816e-03, 1.65840526e-02],
  [-5.59544960e-03, -3.31469356e-02, 4.92826524e-05, -3.58473186e-03, 8.81473816e-03, -4.44667980e-02, -2.12048276e-02],
  [-6.41620131e-03, -5.82894086e-02, -4.21000435e-05, 2.69414564e-02, 1.65840526e-02, -2.12048276e-02, -1.23474124e-01]],

 [[4.85870448e-04, 3.12330658e-04, -4.11775341e-06, 1.44662253e-04, 6.90931607e-04, -7.04368738e-04, -4.10408453e-04],
  [3.12330658e-04, 1.23481002e-04, -6.32179929e-05, 2.60487188e-04, 9.61988122e-03, -1.76830573e-03, -2.34600639e-03],
  [-4.11775341e-06, -6.32179929e-05, 1.93044954e-04, 3.66028286e-05, 1.60263199e-04, -1.49184716e-04, 1.45186671e-05],
  [1.44662253e-04, 2.60487188e-04, 3.66028286e-05, -1.88411555e-05, 1.82330197e-04, -5.74151302e-04, 5.36341249e-04],
  [6.90931607e-04, 9.61988122e-03, 1.60263199e-04, 1.82330197e-04, -2.32142950e-04, 2.81761725e-03, 2.72220434e-03],
  [-7.04368738e-04, -1.76830573e-03, -1.49184716e-04, -5.74151302e-04, 2.81761725e-03, -4.53620477e-03, -1.19177574e-03],
  [-4.10408453e-04, -2.34600639e-03, 1.45186671e-05, 5.36341249e-04, 2.72220434e-03, -1.19177574e-03, -3.65473452e-03]],

 [[1.08825658e-04, -5.10638394e-04, 7.38096611e-04, 2.08578709e-04, 9.61988122e-03, -8.21261404e-04, -8.83686910e-04],
  [-5.10638394e-04, -3.04792293e-03, 8.81225927e-05, 1.70967855e-03, 4.31783543e-02, -1.66527055e-02, -1.51018894e-02],
  [7.38096611e-04, 8.81225927e-05, 8.81991988e-04, 2.61150512e-04, 1.95721515e-04, -7.23893422e-05, 1.49611879e-04],
  [2.08578709e-04, 1.70967855e-03, 2.61150512e-04, -6.00827895e-04, 1.98679691e-04, -4.44643489e-03, 4.46299059e-03],
  [9.61988122e-03, 4.31783543e-02, 1.95721515e-04, 1.98679691e-04, 1.23986184e-03, 1.75433921e-02, 1.78667560e-02],
  [-8.21261404e-04, -1.66527055e-02, -7.23893422e-05, -4.44643489e-03, 1.75433921e-02, -4.00257770e-02, -1.21664983e-02],
  [-8.83686910e-04, -1.51018894e-02, 1.49611879e-04, 4.46299059e-03, 1.78667560e-02, -1.21664983e-02, -3.95184202e-02]],

 [[-2.35842771e-03, 2.27122198e-04, 1.43106614e-04, 4.71592999e-05, 1.60263199e-04, -6.26784906e-05, 7.43045609e-05],
  [2.27122198e-04, 5.70063454e-04, -8.06263340e-05, -7.87096767e-05, 1.95721515e-04, 5.72070762e-04, 5.42159273e-04],
  [1.43106614e-04, -8.06263340e-05, -3.29392663e-04, -2.02580204e-04, 3.07835644e-03, -7.49673180e-04, -9.02108014e-04],
  [4.71592999e-05, -7.87096767e-05, -2.02580204e-04, -1.84068116e-04, -1.87994014e-04, 1.38811570e-04, -2.68956326e-05],
  [1.60263199e-04, 1.95721515e-04, 3.07835644e-03, -1.87994014e-04, 1.79461867e-04, -8.81288897e-06, -1.18006894e-04],
  [-6.26784906e-05, 5.72070762e-04, -7.49673180e-04, 1.38811570e-04, -8.81288897e-06, -1.66372461e-04, 2.28673607e-04],
  [7.43045609e-05, 5.42159273e-04, -9.02108014e-04, -2.68956326e-05, -1.18006894e-04, 2.28673607e-04, 1.94116251e-04]],

 [[4.14071470e-04, 3.71806093e-05, -3.96039814e-06, 4.37747571e-05, 1.82330197e-04, -7.83038941e-05, 4.42423492e-06],
  [3.71806093e-05, 8.56299165e-04, 1.06191075e-04, -5.44750314e-04, 1.98679691e-04, -1.40320465e-03, 1.87375871e-03],
  [-3.96039814e-06, 1.06191075e-04, 5.91580312e-05, -4.83653525e-07, -1.87994014e-04, -8.86596671e-06, -6.22892795e-05],
  [4.37747571e-05, -5.44750314e-04, -4.83653525e-07, -2.91329707e-04, 2.75726697e-03, -1.53309897e-03, -1.37844042e-03],
  [1.82330197e-04, 1.98679691e-04, -1.87994014e-04, 2.75726697e-03, 1.15000550e-04, 1.47765642e-03, -1.35535286e-03],
  [-7.83038941e-05, -1.40320465e-03, -8.86596671e-06, -1.53309897e-03, 1.47765642e-03, -7.91919447e-03, -1.35438964e-04],
  [4.42423492e-06, 1.87375871e-03, -6.22892795e-05, -1.37844042e-03, -1.35535286e-03, -1.35438964e-04, 7.45387169e-03]],

 [[2.76023833e-01, 6.34079525e-02, -1.23646317e-04, -1.62589865e-04, -2.32142950e-04, 1.46173010e-02, 1.46622362e-02],
  [6.34079525e-02, 2.03759837e-01, -8.42075037e-04, 5.33274238e-04, 1.23986184e-03, 8.16714345e-02, 8.14704829e-02],
  [-1.23646317e-04, -8.42075037e-04, 4.88902401e-02, 1.82278307e-04, 1.79461867e-04, 1.20154775e-04, 1.89398340e-04],
  [-1.62589865e-04, 5.33274238e-04, 1.82278307e-04, 4.90249431e-02, 1.15000550e-04, 2.38544801e-02, -2.42297112e-02],
  [-2.32142950e-04, 1.23986184e-03, 1.79461867e-04, 1.15000550e-04, 5.45540437e-02, -2.05408936e-02, -2.03585508e-02],
  [1.46173010e-02, 8.16714345e-02, 1.20154775e-04, 2.38544801e-02, -2.05408936e-02, 1.22812974e-01, 3.55659964e-02],
  [1.46622362e-02, 8.14704829e-02, 1.89398340e-04, -2.42297112e-02, -2.03585508e-02, 3.55659964e-02, 1.23192628e-01]],

 [[-8.61297555e-02, -2.01772593e-02, -2.17873296e-06, -5.18257778e-04, 2.81761725e-03, -4.96801944e-03, -4.86348005e-03],
  [-2.01772593e-02, -7.63569162e-02, 5.33726299e-04, -4.73588291e-03, 1.75433921e-02, -4.62068609e-02, -3.66558200e-02],
  [-2.17873296e-06, 5.33726299e-04, -1.84997600e-02, 1.15699858e-04, -8.81288897e-06, -2.53031026e-04, -2.49235063e-05],
  [-5.18257778e-04, -4.73588291e-03, 1.15699858e-04, -1.88166766e-02, 1.47765642e-03, -1.65047298e-02, 8.81473816e-03],
  [2.81761725e-03, 1.75433921e-02, -8.81288897e-06, 1.47765642e-03, -2.05408936e-02, 1.87447595e-02, 1.47912791e-02],
  [-4.96801944e-03, -4.62068609e-02, -2.53031026e-04, -1.65047298e-02, 1.87447595e-02, -9.60800459e-02, -2.17699284e-02],
  [-4.86348005e-03, -3.66558200e-02, -2.49235063e-05, 8.81473816e-03, 1.47912791e-02, -2.17699284e-02, -6.15173251e-02]],

 [[-8.64325890e-02, -2.05295508e-02, 1.35065949e-04, 6.06447456e-04, 2.72220434e-03, -4.84755835e-03, -4.98734720e-03],
  [-2.05295508e-02, -7.71586463e-02, 1.49986246e-04, 4.96602923e-03, 1.78667560e-02, -3.66410274e-02, -4.56411322e-02],
  [1.35065949e-04, 1.49986246e-04, -1.83002481e-02, 3.78523743e-04, -1.18006894e-04, -4.86193264e-07, -1.30096152e-04],
  [6.06447456e-04, 4.96602923e-03, 3.78523743e-04, -1.90460033e-02, -1.35535286e-03, -8.84816798e-03, 1.65840526e-02],
  [2.72220434e-03, 1.78667560e-02, -1.18006894e-04, -1.35535286e-03, -2.03585508e-02, 1.47912791e-02, 1.87263472e-02],
  [-4.84755835e-03, -3.66410274e-02, -4.86193264e-07, -8.84816798e-03, 1.47912791e-02, -6.14195586e-02, -2.15520335e-02],
  [-4.98734720e-03, -4.56411322e-02, -1.30096152e-04, 1.65840526e-02, 1.87263472e-02, -2.15520335e-02, -9.84962098e-02]],

 [[1.62599008e-01, 1.12105500e-02, -2.79673624e-06, 6.97531989e-04, -7.04368738e-04, 5.01967630e-05, 8.83333973e-04],
  [1.12105500e-02, 5.86693917e-02, -2.94591708e-04, 1.01951371e-03, -8.21261404e-04, 2.16905230e-02, 2.11371107e-02],
  [-2.79673624e-06, -2.94591708e-04, 1.44988464e-02, 6.46947496e-05, -6.26784906e-05, -2.61486504e-05, -7.35524585e-05],
  [6.97531989e-04, 1.01951371e-03, 6.46947496e-05, 1.45087767e-02, -7.83038941e-05, 6.43138835e-03, -5.59544960e-03],
  [-7.04368738e-04, -8.21261404e-04, -6.26784906e-05, -7.83038941e-05, 1.46173010e-02, -4.96801944e-03, -4.84755835e-03],
  [5.01967630e-05, 2.16905230e-02, -2.61486504e-05, 6.43138835e-03, -4.96801944e-03, 2.91791368e-02, 8.62274816e-03],
  [8.83333973e-04, 2.11371107e-02, -7.35524585e-05, -5.59544960e-03, -4.84755835e-03, 8.62274816e-03, 2.73423661e-02]],

 [[4.03556902e-01, 9.34315568e-02, 2.11755748e-04, 2.92987344e-03, -1.76830573e-03, 2.16905230e-02, 2.11836266e-02],
  [9.34315568e-02, 3.28169330e-01, 1.41632573e-03, 2.11500099e-02, -1.66527055e-02, 1.59733404e-01, 1.31352053e-01],
  [2.11755748e-04, 1.41632573e-03, 8.06030855e-02, -4.27228551e-04, 5.72070762e-04, -9.43811034e-06, 3.23194627e-04],
  [2.92987344e-03, 2.11500099e-02, -4.27228551e-04, 8.32311547e-02, -1.40320465e-03, 5.81120153e-02, -3.31469356e-02],
  [-1.76830573e-03, -1.66527055e-02, 5.72070762e-04, -1.40320465e-03, 8.16714345e-02, -4.62068609e-02, -3.66410274e-02],
  [2.16905230e-02, 1.59733404e-01, -9.43811034e-06, 5.81120153e-02, -4.62068609e-02, 2.97462610e-01, 6.77195435e-02],
  [2.11836266e-02, 1.31352053e-01, 3.23194627e-04, -3.31469356e-02, -3.66410274e-02, 6.77195435e-02, 2.01600367e-01]],

 [[-7.50149166e-04, 3.40569537e-04, 2.42887765e-03, -7.90234080e-05, -1.49184716e-04, -2.61486504e-05, -3.93922391e-05],
  [3.40569537e-04, -1.16540139e-03, 1.46794584e-02, -1.55727753e-04, -7.23893422e-05, -9.43811034e-06, -1.36642685e-04],
  [2.42887765e-03, 1.46794584e-02, 8.56563252e-05, 1.20900700e-03, -7.49673180e-04, 5.85980673e-03, 4.54050335e-03],
  [-7.90234080e-05, -1.55727753e-04, 1.20900700e-03, 1.46127019e-04, -8.86596671e-06, 3.02343011e-04, 4.92826524e-05],
  [-1.49184716e-04, -7.23893422e-05, -7.49673180e-04, -8.86596671e-06, 1.20154775e-04, -2.53031026e-04, -4.86193264e-07],
  [-2.61486504e-05, -9.43811034e-06, 5.85980673e-03, 3.02343011e-04, -2.53031026e-04, -1.07368417e-03, 1.06840907e-05],
  [-3.93922391e-05, -1.36642685e-04, 4.54050335e-03, 4.92826524e-05, -4.86193264e-07, 1.06840907e-05, 2.32412353e-04]],

 [[1.11495800e-01, 2.61847750e-02, -1.33872490e-04, 3.07287008e-03, -5.74151302e-04, 6.43138835e-03, 5.69780264e-03],
  [2.61847750e-02, 9.84277485e-02, 6.19213925e-04, 2.02222767e-02, -4.44643489e-03, 5.81120153e-02, 3.29099781e-02],
  [-1.33872490e-04, 6.19213925e-04, 2.37756230e-02, 2.48754383e-04, 1.38811570e-04, 3.02343011e-04, 3.33545782e-05],
  [3.07287008e-03, 2.02222767e-02, 2.48754383e-04, 2.62468253e-02, -1.53309897e-03, 2.67712208e-02, -3.58473186e-03],
  [-5.74151302e-04, -4.44643489e-03, 1.38811570e-04, -1.53309897e-03, 2.38544801e-02, -1.65047298e-02, -8.84816798e-03],
  [6.43138835e-03, 5.81120153e-02, 3.02343011e-04, 2.67712208e-02, -1.65047298e-02, 1.25283792e-01, 2.16572435e-02],
  [5.69780264e-03, 3.29099781e-02, 3.33545782e-05, -3.58473186e-03, -8.84816798e-03, 2.16572435e-02, 4.38469704e-02]],

 [[-8.61297555e-02, -2.01772593e-02, -2.17873296e-06, -5.18257778e-04, 2.81761725e-03, -4.96801944e-03, -4.86348005e-03],
  [-2.01772593e-02, -7.63569162e-02, 5.33726299e-04, -4.73588291e-03, 1.75433921e-02, -4.62068609e-02, -3.66558200e-02],
  [-2.17873296e-06, 5.33726299e-04, -1.84997600e-02, 1.15699858e-04, -8.81288897e-06, -2.53031026e-04, -2.49235063e-05],
  [-5.18257778e-04, -4.73588291e-03, 1.15699858e-04, -1.88166766e-02, 1.47765642e-03, -1.65047298e-02, 8.81473816e-03],
  [2.81761725e-03, 1.75433921e-02, -8.81288897e-06, 1.47765642e-03, -2.05408936e-02, 1.87447595e-02, 1.47912791e-02],
  [-4.96801944e-03, -4.62068609e-02, -2.53031026e-04, -1.65047298e-02, 1.87447595e-02, -9.60800459e-02, -2.17699284e-02],
  [-4.86348005e-03, -3.66558200e-02, -2.49235063e-05, 8.81473816e-03, 1.47912791e-02, -2.17699284e-02, -6.15173251e-02]],

 [[5.05580893e-01, 1.22062781e-01, -1.71585529e-04, 4.73895091e-03, -4.53620477e-03, 2.91791368e-02, 2.75648781e-02],
  [1.22062781e-01, 4.89486454e-01, 5.48978733e-05, 4.98078965e-02, -4.00257770e-02, 2.97462610e-01, 2.01950063e-01],
  [-1.71585529e-04, 5.48978733e-05, 1.17109249e-01, -4.07503249e-04, -1.66372461e-04, -1.07368417e-03, -1.35019138e-04],
  [4.73895091e-03, 4.98078965e-02, -4.07503249e-04, 1.26360985e-01, -7.91919447e-03, 1.25283792e-01, -4.44667980e-02],
  [-4.53620477e-03, -4.00257770e-02, -1.66372461e-04, -7.91919447e-03, 1.22812974e-01, -9.60800459e-02, -6.14195586e-02],
  [2.91791368e-02, 2.97462610e-01, -1.07368417e-03, 1.25283792e-01, -9.60800459e-02, 7.33901219e-01, 1.29071321e-01],
  [2.75648781e-02, 2.01950063e-01, -1.35019138e-04, -4.44667980e-02, -6.14195586e-02, 1.29071321e-01, 3.22752496e-01]],

 [[1.57539641e-01, 3.71299836e-02, -6.75780472e-05, -6.16448449e-05, -1.19177574e-03, 8.62274816e-03, 8.58054677e-03],
  [3.71299836e-02, 1.41292648e-01, -8.09036149e-05, -2.17599478e-05, -1.21664983e-02, 6.77195435e-02, 6.79235993e-02],
  [-6.75780472e-05, -8.09036149e-05, 3.42969014e-02, -1.65981732e-04, 2.28673607e-04, 1.06840907e-05, 1.63705951e-04],
  [-6.16448449e-05, -2.17599478e-05, -1.65981732e-04, 3.51531143e-02, -1.35438964e-04, 2.16572435e-02, -2.12048276e-02],
  [-1.19177574e-03, -1.21664983e-02, 2.28673607e-04, -1.35438964e-04, 3.55659964e-02, -2.17699284e-02, -2.15520335e-02],
  [8.62274816e-03, 6.77195435e-02, 1.06840907e-05, 2.16572435e-02, -2.17699284e-02, 1.29071321e-01, 3.44269148e-02],
  [8.58054677e-03, 6.79235993e-02, 1.63705951e-04, -2.12048276e-02, -2.15520335e-02, 3.44269148e-02, 1.29822751e-01]],

 [[4.00351158e-06, 3.10968983e-02, 6.49858828e-07, -7.18589381e-04, -4.10408453e-04, 8.83333973e-04, 7.23376172e-03],
  [3.10968983e-02, 5.76087094e-02, 1.47309077e-04, -9.57952552e-04, -8.83686910e-04, 2.11836266e-02, 2.18692780e-02],
  [6.49858828e-07, 1.47309077e-04, 1.45160040e-02, -4.67336341e-05, 7.43045609e-05, -3.93922391e-05, -4.29018115e-05],
  [-7.18589381e-04, -9.57952552e-04, -4.67336341e-05, 1.45732653e-02, 4.42423492e-06, 5.69780264e-03, -6.41620131e-03],
  [-4.10408453e-04, -8.83686910e-04, 7.43045609e-05, 4.42423492e-06, 1.46622362e-02, -4.86348005e-03, -4.98734720e-03],
  [8.83333973e-04, 2.11836266e-02, -3.93922391e-05, 5.69780264e-03, -4.86348005e-03, 2.75648781e-02, 8.58054677e-03],
  [7.23376172e-03, 2.18692780e-02, -4.29018115e-05, -6.41620131e-03, -4.98734720e-03, 8.58054677e-03, 2.90661444e-02]],

 [[4.01806338e-01, 9.26345761e-02, 4.29247978e-05, -2.83653712e-03, -2.34600639e-03, 2.11371107e-02, 2.18692780e-02],
  [9.26345761e-02, 3.27331193e-01, 1.11908108e-03, -2.08755834e-02, -1.51018894e-02, 1.31352053e-01, 1.59988542e-01],
  [4.29247978e-05, 1.11908108e-03, 8.00035122e-02, -3.29573637e-04, 5.42159273e-04, -1.36642685e-04, -4.39298987e-04],
  [-2.83653712e-03, -2.08755834e-02, -3.29573637e-04, 8.30857196e-02, 1.87375871e-03, 3.29099781e-02, -5.82894086e-02],
  [-2.34600639e-03, -1.51018894e-02, 5.42159273e-04, 1.87375871e-03, 8.14704829e-02, -3.66558200e-02, -4.56411322e-02],
  [2.11371107e-02, 1.31352053e-01, -1.36642685e-04, 3.29099781e-02, -3.66558200e-02, 2.01950063e-01, 6.79235993e-02],
  [2.18692780e-02, 1.59988542e-01, -4.39298987e-04, -5.82894086e-02, -4.56411322e-02, 6.79235993e-02, 2.99258159e-01]],

 [[1.29827165e-03, -2.32825687e-04, 2.46156102e-03, -9.85978867e-05, 1.45186671e-05, -7.35524585e-05, -4.29018115e-05],
  [-2.32825687e-04, -1.07926173e-03, 1.40652329e-02, 2.09686835e-04, 1.49611879e-04, 3.23194627e-04, -4.39298987e-04],
  [2.46156102e-03, 1.40652329e-02, 2.88882745e-04, -1.13841056e-03, -9.02108014e-04, 4.54050335e-03, 5.82326513e-03],
  [-9.85978867e-05, 2.09686835e-04, -1.13841056e-03, 1.19490330e-04, -6.22892795e-05, 3.33545782e-05, -4.21000435e-05],
  [1.45186671e-05, 1.49611879e-04, -9.02108014e-04, -6.22892795e-05, 1.89398340e-04, -2.49235063e-05, -1.30096152e-04],
  [-7.35524585e-05, 3.23194627e-04, 4.54050335e-03, 3.33545782e-05, -2.49235063e-05, -1.35019138e-04, 1.63705951e-04],
  [-4.29018115e-05, -4.39298987e-04, 5.82326513e-03, -4.21000435e-05, -1.30096152e-04, 1.63705951e-04, -2.31729058e-04]],

 [[-1.10880675e-01, -2.59488203e-02, -3.60941579e-05, 3.26859611e-03, 5.36341249e-04, -5.59544960e-03, -6.41620131e-03],
  [-2.59488203e-02, -9.96812292e-02, 6.39578084e-04, 1.91463504e-02, 4.46299059e-03, -3.31469356e-02, -5.82894086e-02],
  [-3.60941579e-05, 6.39578084e-04, -2.37961138e-02, 2.42038461e-04, -2.68956326e-05, 4.92826524e-05, -4.21000435e-05],
  [3.26859611e-03, 1.91463504e-02, 2.42038461e-04, -2.64198829e-02, -1.37844042e-03, -3.58473186e-03, 2.69414564e-02],
  [5.36341249e-04, 4.46299059e-03, -2.68956326e-05, -1.37844042e-03, -2.42297112e-02, 8.81473816e-03, 1.65840526e-02],
  [-5.59544960e-03, -3.31469356e-02, 4.92826524e-05, -3.58473186e-03, 8.81473816e-03, -4.44667980e-02, -2.12048276e-02],
  [-6.41620131e-03, -5.82894086e-02, -4.21000435e-05, 2.69414564e-02, 1.65840526e-02, -2.12048276e-02, -1.23474124e-01]],

 [[-8.64325890e-02, -2.05295508e-02, 1.35065949e-04, 6.06447456e-04, 2.72220434e-03, -4.84755835e-03, -4.98734720e-03],
  [-2.05295508e-02, -7.71586463e-02, 1.49986246e-04, 4.96602923e-03, 1.78667560e-02, -3.66410274e-02, -4.56411322e-02],
  [1.35065949e-04, 1.49986246e-04, -1.83002481e-02, 3.78523743e-04, -1.18006894e-04, -4.86193264e-07, -1.30096152e-04],
  [6.06447456e-04, 4.96602923e-03, 3.78523743e-04, -1.90460033e-02, -1.35535286e-03, -8.84816798e-03, 1.65840526e-02],
  [2.72220434e-03, 1.78667560e-02, -1.18006894e-04, -1.35535286e-03, -2.03585508e-02, 1.47912791e-02, 1.87263472e-02],
  [-4.84755835e-03, -3.66410274e-02, -4.86193264e-07, -8.84816798e-03, 1.47912791e-02, -6.14195586e-02, -2.15520335e-02],
  [-4.98734720e-03, -4.56411322e-02, -1.30096152e-04, 1.65840526e-02, 1.87263472e-02, -2.15520335e-02, -9.84962098e-02]],

 [[1.57539641e-01, 3.71299836e-02, -6.75780472e-05, -6.16448449e-05, -1.19177574e-03, 8.62274816e-03, 8.58054677e-03],
  [3.71299836e-02, 1.41292648e-01, -8.09036149e-05, -2.17599478e-05, -1.21664983e-02, 6.77195435e-02, 6.79235993e-02],
  [-6.75780472e-05, -8.09036149e-05, 3.42969014e-02, -1.65981732e-04, 2.28673607e-04, 1.06840907e-05, 1.63705951e-04],
  [-6.16448449e-05, -2.17599478e-05, -1.65981732e-04, 3.51531143e-02, -1.35438964e-04, 2.16572435e-02, -2.12048276e-02],
  [-1.19177574e-03, -1.21664983e-02, 2.28673607e-04, -1.35438964e-04, 3.55659964e-02, -2.17699284e-02, -2.15520335e-02],
  [8.62274816e-03, 6.77195435e-02, 1.06840907e-05, 2.16572435e-02, -2.17699284e-02, 1.29071321e-01, 3.44269148e-02],
  [8.58054677e-03, 6.79235993e-02, 1.63705951e-04, -2.12048276e-02, -2.15520335e-02, 3.44269148e-02, 1.29822751e-01]],

 [[4.98928733e-01, 1.21837868e-01, 2.59899668e-04, -4.70502623e-03, -3.65473452e-03, 2.73423661e-02, 2.90661444e-02],
  [1.21837868e-01, 4.89352274e-01, -4.05768050e-04, -4.93590776e-02, -3.95184202e-02, 2.01600367e-01, 2.99258159e-01],
  [2.59899668e-04, -4.05768050e-04, 1.17020219e-01, 2.09471598e-04, 1.94116251e-04, 2.32412353e-04, -2.31729058e-04],
  [-4.70502623e-03, -4.93590776e-02, 2.09471598e-04, 1.27312815e-01, 7.45387169e-03, 4.38469704e-02, -1.23474124e-01],
  [-3.65473452e-03, -3.95184202e-02, 1.94116251e-04, 7.45387169e-03, 1.23192628e-01, -6.15173251e-02, -9.84962098e-02],
  [2.73423661e-02, 2.01600367e-01, 2.32412353e-04, 4.38469704e-02, -6.15173251e-02, 3.22752496e-01, 1.29822751e-01],
  [2.90661444e-02, 2.99258159e-01, -2.31729058e-04, -1.23474124e-01, -9.84962098e-02, 1.29822751e-01, 7.34495599e-01]]]]

#6. Solve F'C' = C'e for C' and e, then transform C' -> C -> P. Re-formulate F (the J and K integrals) using P, solve for C' and e again and repeat until e converges. Also add nuclear repulsion energy to E_0.

In [4]:
# Calculate nuclear repulsion energy

E_nuc = 0.0
for i in range(len(Molecule["Atom_label"])):
    for j in range(i + 1, len(Molecule["Atom_label"])):
        Z1, Z2 = Molecule['Z'][i], Molecule['Z'][j]
        R = np.sqrt((Molecule['x'][i] - Molecule['x'][j])**2 +
                    (Molecule['y'][i] - Molecule['y'][j])**2 +
                    (Molecule['z'][i] - Molecule['z'][j])**2)
        E_nuc += 0.5 * Z1 * Z2 / R
print("Nuclear Repulsion Energy =", E_nuc, "Hartrees")

# N.B. Factor of 0.5 to avoid double counting

# Find J, K and F, and Transform F -> F' = X^T F X

J = np.zeros((len(basis_set), len(basis_set)))
K = np.zeros((len(basis_set), len(basis_set)))

def J_and_K_finder(P, eri, basis_set):
  for i in range(len(basis_set)):
    for j in range(len(basis_set)):
      for k in range(len(basis_set)):
        for l in range(len(basis_set)):
          J[i, j] += P[k, l] * eri[i, j, k, l]
          K[i, j] += P[k, l] * eri[i, k, j, l]
  return J, K

F = T + V + J - 0.5 * K

# Setup HFSCF iterations which will print E_0

def SCF_iterator(F, X, T, V, basis_set, n_occ):

  F_prime = X.T @ F @ X

  # Obtain eigenvectors (C') of the problem: (F' - e)(C') = 0
  e = sp.symbols('e')
  F_prime_less_e = F_prime - np.diag([e] * len(basis_set))
  eigenenergies, C_prime = np.linalg.eig(F_prime)
  print("E_0 =", min(eigenenergies.real) + E_nuc, "Hartrees")

  # Convert C' into C. N.B. That our basis functions are not orthogonal, so we obtain C via:
  # C = (X^T)^-1 C' X^-1 = X^-1 C' X^-1; assuming X is symmetric.
  X_inv = np.linalg.inv(X)
  C = X_inv @ C_prime @ X_inv
  C = np.array(C)
  C_occ = C[:, :n_occ]

  # Find new P matrix
  P = np.zeros((len(basis_set), len(basis_set)))
  for i in range(n_occ):
    for mu in range(len(basis_set)):
      for nu in range(len(basis_set)):
        P[mu, nu] += 2*(C_occ[mu, i] * C_occ[nu, i])

  # Call new J and K functions (T, V, S, X do not change throughout SCF iterations)
  J, K = demo(P, eri, basis_set)

  F_new = np.zeros((len(basis_set), len(basis_set)))
  for i in range(len(basis_set)):
    for j in range(len(basis_set)):
      F_new[i, j] = T[i, j] + V[i, j] + J[i, j] - 0.5 * K[i, j]
  return P, F_new

for i in range(50):
  P, F = SCF_iterator(F, X, T, V, basis_set, n_occ)

NameError: name 'Molecule' is not defined